In [2]:
#%pip install ipywidgets
#%pip install voila
#%pip install voila-gridstack
### 

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import pandas.util.testing as tm
import statsmodels.api as sm
import seaborn as sns
import statsmodels.formula.api as smf
from IPython.display import display
from IPython.display import clear_output
import itertools
import ipywidgets as widgets

In [4]:
### input data
## data filename
datafilename1 = 'sepsis_survival_primary.csv'
datafilename2 = 'sepsis_survival_study.csv'
datafilename3 = 'sepsis_survival_validation.csv'

## depended Variable column name
dependVar = 'hospital_outcome_1alive_0dead'
## exclude columns from analyze (text, urls, etc. )
excludeColumns = []
## minimum correlation coeff to assume as a key variable
minimumCorrCoef = 0.01
threshold = 0.5


In [5]:
## read CSV file, autodetect delimeters, skip spaces in names
df1 = pd.read_csv(datafilename1, sep=None, engine="python", skipinitialspace=True)
df2 = pd.read_csv(datafilename2, sep=None, engine="python", skipinitialspace=True)
df3 = pd.read_csv(datafilename3, sep=None, engine="python", skipinitialspace=True)
dfs = [df1, df2, df3]
df1.head()

,age_years,sex_0male_1female,episode_number,hospital_outcome_1alive_0dead
0,21,1,1,1
1,20,1,1,1
2,21,1,1,1
3,77,0,1,1
4,72,0,1,1


In [6]:
for df in dfs:
    ## exclude columns
    df.drop(excludeColumns, axis ='columns', inplace = True)
    ## and drop  all NaN
    df.dropna(inplace=True)
    ## place depended var into pos 0
    poped = df.pop(dependVar)
    df.insert(0, poped.name, poped)
        
## independed variable columns names  
independVarList = list(df.columns.values)
independVarList.remove(dependVar)

In [7]:
class LogisticRegression:
    def __init__(self, dfTrain, dfTest):
        self.dfTrain = dfTrain.copy()
        self.dfTest = dfTest.copy()
        self.models = []
        self.modresults = []
        self.predictVars = []
        ### make models for all variables and combinations
        self.allResults = pd.DataFrame(columns=['vars', 'passprob', 'res', 'model', 'BA', 'TP', 'FN', 'TN', 'FP'])
        self.CalculateModels()
        self.TestModels()
        
    def CalculateModels(self):
        keyVars = self.GetKeyVariables(self.dfTrain)
        self.dependVar = keyVars['key'].iloc[0]
        ### All vars into list.
        self.predictVars = keyVars['key'].iloc[1:].to_list()    ## .iloc[1:NvarsCount+1].to_list()
        ### make models for all variables, sequently adding one by one
        for n,indvar in enumerate(self.predictVars):
            indvars = self.predictVars[:n+1]
            modvars = list(map(lambda orig_string: 'Q("' + orig_string + '")', indvars))
            model = smf.logit(formula = '' + dependVar + ' ~ ' + ' + '.join(modvars), data = self.dfTrain)
            res = model.fit(disp=False)
            self.models.append(model)
            self.modresults.append(res)


    def GetKeyVariables(self, df):
        parCorr = pd.DataFrame(df.corr() )
        n = len(parCorr.columns)
        keyVars = pd.DataFrame(columns=['key', 'val'])
        ## depended variable moved to index 0
        i = 0
        for j in range( n):
            if j >= i:
                keyVars = keyVars.append({'key':parCorr.columns[j],'val':parCorr.iloc[i, j]}, ignore_index=True)
        ## sort key vars by value
        ##### keyVars.pop(df.columns[0])
        keyVars.sort_values(by='val', key=abs, ascending=False, inplace=True)
        return keyVars

    
    def get_BA(self, crossdf):
        crossdf.index = crossdf.index.map(str)
        crossdf.columns = crossdf.columns.map(str)
        try: TN = crossdf.loc['0','0']
        except: TN = 0
        try: FN = crossdf.loc['0','1']
        except: FN = 0
        try: FP = crossdf.loc['1','0']
        except: FP = 0
        try: TP = crossdf.loc['1','1']
        except: TP = 0
        ###print('tn=', TN, 'fn=', FN, 'fp=', FP, 'tp=', TP)
        # sensitivity = (TP)/(TP+FN)
        # specificity = (TN)/(TN+FP)
        # precision = (TP)/(TP+FP)
        try: sensitivity = TP/(TP + FN)
        except: sensitivity = np.NaN
        try: specificity = TN/(TN + FP)
        except: specificity = np.NaN
        try: precision = TP/(TP + FP)
        except: precision = np.NaN
        BA = (sensitivity + specificity)/2
        return dict(zip(['BA', 'sensitivity', 'specificity', 'precision', 'TN', 'FN', 'TP', 'FP'], [BA, sensitivity, specificity, precision, TN, FN, TP, FP]))
        pass
    
    
    def TestModels(self):
        ### calculate BA for all threashholds...
        passProbabilities = np.linspace(0,1,int(1/0.05)+1)

        ## Generate all possible combinations of variables
        variableList = sum([list(map(list, itertools.combinations(self.predictVars, i))) for i in range(len(self.predictVars) + 1)], [])
        variableList = list(filter(None, variableList))

        for n,indvars in enumerate(variableList):
            ## progress # display(indvars)
            modvars = list(map(lambda orig_string: 'Q("' + orig_string + '")', indvars))
            model = smf.logit(formula = '' + dependVar + ' ~ ' + ' + '.join(modvars), data = self.dfTrain )
            fitresult = model.fit(disp=False)
            for prob in passProbabilities:
                inSample = pd.DataFrame({'probability':fitresult.predict(self.dfTest[indvars])}) 
                inSample['Model prediction'] = (inSample['probability'] >= prob).astype(int) ## 0 or 1 values based on probabilities
                confMatrix = pd.crosstab(inSample['Model prediction'], self.dfTest[self.dependVar], dropna=False)
                ba = self.get_BA(confMatrix)
                ### add all info to one dataframe
                ## TODO: memory consumption?
                self.allResults = self.allResults.append({'vars':indvars, 
                                   'passprob':prob, 
                                   'res':fitresult, 
                                   'model':model, **(ba)},
                                 ignore_index=True)        

        
    def ShowResultsTable(self):
        def _ui_show_result_table(sortcol1,sortcol2, sortorder1, sortorder2):
            print("Selected columns and order: ", sortcol1,sortcol2, sortorder1, sortorder2)
            allResults = self.allResults.reindex(
                            index = self.dfBA.sort_values(by=[sortcol1,sortcol2], #['TP', 'BA'],#
                                        ascending=[sortorder1,sortorder2]).index,
                            copy=True)
            pd.set_option("display.max_rows", 400)
            pd.set_option("display.max_columns", 10)
            pd.set_option("display.max_colwidth", 2000)
            display(allResults.loc[:, ~allResults.columns.isin(['res', 'model'])])
            
        sortableColumns = [item for item in list(self.allResults.columns) 
                   if item not in list(['vars', 'res', 'model'])] 
        self.dfBA = self.allResults.loc[:, sortableColumns].astype('float64').round(decimals=1)
        sort1col = widgets.SelectionSlider(
            options=sortableColumns,
            value='BA',
            description='Sort by...',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True
        )
        sort2col = widgets.SelectionSlider(
            options=sortableColumns,
            value='passprob',
            description='and by...',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True
        )
        sort1order = widgets.ToggleButtons(
            options=[('Ascending',True), ('Descending',False)],
            value=False,
            description='Order:',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltips=['Sort column in ascending order (1..99)', 'Sort column in descending order (99..1)']
        )
        sort2order = widgets.ToggleButtons(
            options=[('Ascending',True), ('Descending',False)],
            value=True,
            description='Order:',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltips=['Sort column in ascending order (1..99)', 'Sort column in descending order (99..1)']
        )

        sort1ui = widgets.HBox([sort1col,sort1order])
        sort2ui = widgets.HBox([sort2col,sort2order])
        resultui = widgets.interactive_output(_ui_show_result_table,
                            {'sortcol1':sort1col,'sortcol2':sort2col, 
                             'sortorder1':sort1order,'sortorder2':sort2order})
        display(sort1ui,sort2ui,resultui)
        
    def getModelPrediction(self, modelNumber, dfInput):
        modelresult = self.allResults.iloc[modelNumber].res
        return modelresult.predict(dfInput)


In [8]:
logreg1 = LogisticRegression(df1, df2)  ## df1-big as train
logreg3 = LogisticRegression(df3, df2)  ## df3-small as train

In [9]:
logreg1 = LogisticRegression(df1, df2)
logreg1.ShowResultsTable()

Output()

In [10]:
def histogramPlot(df):    
    allVars=df.columns
    nxdr = wdg.Dropdown(options = allVars, value = allVars[1], 
                        description = 'X value', option = 'X var')

    nBinsdr = wdg.Dropdown(options=[2,10,15,20,40], value=2, 
                        description = 'Bins number', option = 'Bin value')

    def plot_histogram(x = 'age_years', nBins = 15, PlotKDE = False):    
        sns.histplot(data = df[x] , bins = nBins, kde = PlotKDE)

    out = wdg.interact(plot_histogram, nBins = nBinsdr,PlotKDE = False,x=nxdr)
    ui = wdg.HBox([nxdr,nBinsdr])

In [11]:
def scatterPlot(df):
    allVars=df.columns
    nxdr2x = wdg.Dropdown(options = allVars, value = allVars[1], 
                        description = 'X value', option = 'X var')
    nxdr2y = wdg.Dropdown(options = allVars, value = allVars[1], 
                        description = 'X value', option = 'X var')

    def plot_scatterplot(x = 'age_years', y = 'hospital_outcome_1alive_0dead'):    
        sns.scatterplot(data = df, x = x, y = y)

    out2 = wdg.interact(plot_scatterplot, x=nxdr2x, y = nxdr2y)
    ui = wdg.HBox([nxdr2x,nxdr2y])


In [12]:
#scatterPlot(df1)

## Regular regressional analise through all possible variables

In [13]:
def HeatmapCorr(df): 
    sns.set(rc = {'figure.figsize':(15,8)})
    sns.set_theme(style="white")
    corr = df.corr()
    # Generate a mask for the upper triangle
    mask = np.triu(np.ones_like(corr, dtype=bool))
    # Custom colormap
    cmap = sns.diverging_palette(275, 150, s=90, l=50, n=9, as_cmap=True)
    sns.heatmap(corr, mask=mask, cmap=cmap,  linewidths=0.3, cbar_kws={"shrink":0.5})

In [14]:
#HeatmapCorr(df3)

In [36]:
class OutcomePredictionUI: 
    def __init__(self, logReg, independentVars):
        self.independentVars = independentVars.copy()
        self.logReg = logReg
        # self.formdf = pd.DataFrame(columns= [*self.independentVars, 'Result probability'])
        self.formdf = self.logReg.dfTrain.drop(self.logReg.dfTrain.index)
        self.i,self.j = 4,4
        self.grid = widgets.GridspecLayout(self.i,self.j)
        ### self.dfval = df[self.independentVars]
        self.widgetsCreate()

    def create_expanded_button(self,description, button_style):
        return widgets.Button(description=description, button_style=button_style,
                          layout=widgets.Layout(height='auto', width='auto'))

    def widgetsCreate(self):
        #BUTTONS
        self.btn_Send = self.grid[2, 0] = self.create_expanded_button('Add to date frame'.format(0, 0), 'warning')
        self.btn_Clear = self.grid[2, 1] = self.create_expanded_button('Clear'.format(0, 0), 'warning')
        self.btn_Results = self.grid[2, 2] = self.create_expanded_button('Get results'.format(0, 0), 'warning')
        #Fields to select
        self.btn_sex = self.grid[0,0] = widgets.Dropdown(description='Sex', options =['Female','Male'])
        self.btn_age = self.grid[0,1] = widgets.Dropdown(description='Age', options =range(26,100))
        self.btn_episodes = self.grid[0,2] = widgets.Dropdown(description='Episodes number', options =range(1,5))
        #Outcome
        #self.btn_outcome = self.grid[1,0] = wdg.Text(value='.......', description='Outcome')
        
        # when button is clicked
        for j in range(3):
            self.grid[2,j].on_click(self.on_btn_click)
        #display the grid
        display(self.grid)

    def addToDF(self):
        
        print('Age',self.btn_age.value,
              ', Sex',self.btn_sex.value,
              ', Episode Number',self.btn_episodes.value)
        
        #sex_value = 1
        if self.btn_sex.value == 'Female':
            self.sex_value = 1
        else:
            self.sex_value = 0
        
        self.formdf = self.formdf.append(pd.DataFrame([[self.btn_age.value,self.btn_sex.value, 
                                           self.btn_episodes.value]], columns = self.independentVars),
                                         ignore_index=True)
        clear_output(wait=True)
        self.widgetsCreate()
        #display(self.formdf)
        #return self.formdf
        
    def convertDF(self):
        for i in range(0,len(self.formdf['sex_0male_1female'])):
            self.formdf['sex_0male_1female'] =self.formdf['sex_0male_1female'].replace(['Male'],0)
            self.formdf['sex_0male_1female'] =self.formdf['sex_0male_1female'].replace(['Female'],1)
            '''
            if self.formdf['sex_0male_1female'][i] == 'Female':
                self.formdf['sex_0male_1female'].iloc[i] = 1
            elif self.formdf['sex_0male_1female'][i] == 'Male':
                self.formdf['sex_0male_1female'].iloc[i] = 0 
            else:
                print('nothign to convery')
            '''
        #self.formdf['sex_0male_1female'][i] = (self.formdf['sex_0male_1female'][i]=='Male').astype(int)
        #self.formdf  
        
    def clear(self):
        clear_output(wait=True)
        #self.formdf = pd.DataFrame(columns= self.independentVars)
        self.formdf = self.logReg.dfTrain.drop(self.logReg.dfTrain.index)
        self.widgetsCreate()

    def on_btn_click(self, btn):
        if btn.description == 'Add to date frame':
            self.addToDF()
        elif btn.description == 'Clear':
            self.clear()
        elif btn.description == 'Get results':
            self.convertDF()
            #display(self.formdf)
            resProbs = self.logReg.getModelPrediction(2, self.formdf)
            self.formdf['Result probability'] = resProbs
        self.show_results()

    
    def show_results(self):
        display(self.formdf)

In [37]:
OutcomePredictionUI(logreg1, ["age_years", "sex_0male_1female", "episode_number"] )

GridspecLayout(children=(Button(button_style='warning', description='Add to date frame', layout=Layout(grid_ar…

,hospital_outcome_1alive_0dead,age_years,sex_0male_1female,episode_number
0,NaN,26,Female,1
1,NaN,26,Female,1
2,NaN,26,Female,1
3,NaN,26,Female,1
4,NaN,26,Male,1
5,NaN,26,Male,1
6,NaN,26,Male,1


,hospital_outcome_1alive_0dead,age_years,sex_0male_1female,episode_number,Result probability
0,NaN,26,1,1,0.989093
1,NaN,26,1,1,0.989093
2,NaN,26,1,1,0.989093
3,NaN,26,1,1,0.989093
4,NaN,26,0,1,0.989093
5,NaN,26,0,1,0.989093
6,NaN,26,0,1,0.989093


In [17]:
     """
        Alist = np.array([btn_age.value,sex_value,btn_episodes.value])
        print(Alist)

        Alist_scaled = (Alist - dfval.min())/(dfval.max() - dfval.min())   
        print(Alist_scaled)
        x = Alist_scaled
        output_vals = np.dot(x,parameters_out["weight"]) + parameters_out["bias"]
        predictions = sigmoid(output_vals) >= 1/2
        if predictions == True:
             btn_outcome.value = 'ALIVE'
        elif predictions ==False:
            btn_outcome.value = 'DEAD'
        """

'\n   Alist = np.array([btn_age.value,sex_value,btn_episodes.value])\n   print(Alist)\n\n   Alist_scaled = (Alist - dfval.min())/(dfval.max() - dfval.min())   \n   print(Alist_scaled)\n   x = Alist_scaled\n   output_vals = np.dot(x,parameters_out["weight"]) + parameters_out["bias"]\n   predictions = sigmoid(output_vals) >= 1/2\n   if predictions == True:\n        btn_outcome.value = \'ALIVE\'\n   elif predictions ==False:\n       btn_outcome.value = \'DEAD\'\n   '